# Statistics

**Instalation of the packages**

In [ ]:
pip install "statsmodels"

## Read and merge files

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statistics
import re
import statsmodels.api as sm
import matplotlib.pyplot as plt
import glob

## Read and prepare the files from ALPS

In [ ]:
files = glob.glob(r"C:\Users\juank\OneDrive\Desktop\Master_code\ALPS\HRV\Sub1\*.csv")

Sub1 = pd.DataFrame()
for f in files:
    csv = pd.read_csv(f)
    csv["Unit"] = csv["Unit"].fillna(" ")
    csv["Variables"] = csv["Variable"] + " " + csv["Unit"]
    Label = csv.Label[1]
    File = "FileName"
    csv.rename(columns={"Value":Label}, inplace= True)
    csv = csv.drop(["Signal","Method","Unit","Variable","Session","Dataset","Label","Subject"],axis=1)
    csv = csv.set_index("Variables")
    csv = csv.T
    Sub1 = Sub1.append(csv)
#merge all the files created in ALPS per subject into only one df

In [ ]:
Sub1.index.names = ["FileName"] #set an index

In [ ]:
files = glob.glob(r"C:\Users\juank\OneDrive\Desktop\Master_code\ALPS\HRV\Sub2\*.csv")

Sub2 = pd.DataFrame()
for f in files:
    csv = pd.read_csv(f)
    csv["Unit"] = csv["Unit"].fillna(" ")
    csv["Variables"] = csv["Variable"] + " " + csv["Unit"]
    Label = csv.Label[1]
    File = "FileName"
    csv.rename(columns={"Value":Label}, inplace= True)
    csv = csv.drop(["Signal","Method","Unit","Variable","Session","Dataset","Label","Subject"],axis=1)
    csv = csv.set_index("Variables")
    csv = csv.T
    Sub2 = Sub2.append(csv)
#merge all the files created in ALPS per subject into only one df

In [ ]:
Sub2.index.names = ["FileName"] #set an index

In [ ]:
df_ALPS = pd.concat([Sub1,Sub2,Sub3,Sub4], axis = 0) # merge al dfs

In [ ]:
df_ALPS #visualize

In [ ]:
df_ALPS = df_ALPS.set_axis(['S1_VLFpeak_FFT Hz', 'S1_VLFpow_FFT ms2', 'S1_VLFpow_FFT log', "S1_VLFpow_FFT %",
                 "S1_LFpeak_FFT Hz",'S1_LFpow_FFT ms2','S1_LFpow_FFT log','S1_LFpow_FFT %','S1_LFpow_FFT n.u.',
                  'S1_HFpeak_FFT Hz','S1_HFpow_FFT ms2','S1_HFpow_FFT log','S1_HFpow_FFT %','S1_HFpow_FFT n.u.',
                  'S1_LF_HF_ratio_FFT','S1_SampEn','S1_Mean RR ms','S1_Mean HR bpm','S1_Max HR bpm','S1_Min HR bpm',
                 'S1_SD HR bpm','S1_SDNN ms','S1_SDNNI ms','S1_SDANN ms','S1_RMSSD ms','S1_NNxx beats',
                  'S1_pNNxx %', 'S1_HRV triangular index','S1_TINN ms'], axis='columns')
#change the axis of the df

## Read and prepare the files from KUBIOS

In [ ]:
Sub1k = pd.read_csv("path...\KubiosHRVresultsSub1.csv", header = 1) #read data file
Sub2k = pd.read_csv("path...\KubiosHRVresultsSub2.csv", header = 1) #read data file
Sub3k = pd.read_csv("path...\KubiosHRVresultsSub3.csv", header = 1) #read data file
#... add all files from Kubios HRV

In [ ]:
df_kubios = pd.concat([Sub1k,Sub2k,Sub3k,Sub4k], axis = 0) # merge all files

In [ ]:
cols = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
df_kubios = df_kubios.drop(df_kubios.columns[cols],axis=1) # eliminate informational columns

In [ ]:
df_kubios.drop(columns=["Unnamed: 103"], inplace=True) # eliminate "random" column

In [ ]:
cols = [1,2,3,31,33, 34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,
        66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82]
df_kubios = df_kubios.drop(df_kubios.columns[cols],axis=1) # eliminate informational columns

In [ ]:
df_kubios = df_kubios.set_index("FileName") #set the ne indices

In [ ]:
df_kubios = df_kubios.filter(like = "Empatica", axis = 0)

In [ ]:
df_kubios = df_kubios.astype(float)
df_kubios.rename(columns=lambda x: x.replace('(',''), inplace=True)
df_kubios.rename(columns=lambda x: x.replace(')',''), inplace=True)  #delete RE expresions

In [ ]:
df_kubios #visualize

In [ ]:
df1 = df_kubios.astype(float)
df2 = df_ALPS.astype(float) # transform all our values to floats

## Alter the label of our datasets

In [ ]:
df1.rename(index=lambda x: x.replace('Sub1_Empatica_Segment','1_'), inplace=True)
df1.rename(index=lambda x: x.replace('Sub2_Empatica_Segment','2_'), inplace=True)
df1.rename(index=lambda x: x.replace('Sub3_Empatica_Segment','3_'), inplace=True)

df1.rename(index=lambda x: x.replace('.csv',''), inplace=True)
# edit the beginning of the index of our df's, depends of how was save in the files

In [ ]:
df1 #visualize

## Statistical Tests

### Mean Difference - Global

In [ ]:
intro = pd.Index(df_kubios.columns) #create index for iteration process

In [ ]:
data = []
data1 = []
data2 = [] #create empty lists to save the data
for column in df_kubios.columns: #mean difference between the complete datasets
        x = df_kubios[column].astype(float).mean() #list the means of Empatica
        y = df_ALPS[column].astype(float).mean() #list the means of Faros
        z = (df_kubios[column].astype(float).mean() - df_ALPS[column].astype(float).mean()) #list the difference of the means
        z = abs (z)
        data.append (x)
        data1.append (y)
        data2.append (z)
mean_x = pd.DataFrame (data, index = intro, columns=["Means Kubios"])
mean_y = pd.DataFrame (data1, index = intro, columns=["Means ALPS"])
mean_z = pd.DataFrame (data2, index = intro, columns=["Mean Difference"])
#generate df's
mean_diff = pd.concat ([mean_x,mean_y,mean_z], axis = 1) #concat our 3 datasets
mean_diff.index.name = "Name" #give name to the index
mean_diff#visualize

In [ ]:
df1 #visualize

In [ ]:
df2 #visualize

In [ ]:
df12_merged = df1.merge(df2, suffixes=[" _kubios", " _ALPS"], on="FileName") # merge the datasets

In [ ]:
df12_merged #visualize

Fix the names of our columns (remove "(" and ")")

In [ ]:
df12_merged.rename(columns=lambda x: x.replace('(',''), inplace=True)
df12_merged.rename(columns=lambda x: x.replace(')',''), inplace=True) 

Create our working list for itemize in the loop

In [ ]:
w = df_Emp #copy one of the original datasets

In [ ]:
w.rename(columns=lambda x: x.replace('(',''), inplace=True)
w.rename(columns=lambda x: x.replace(')',''), inplace=True) #remove the "(" and ")" 's

In [ ]:
column_names = w.columns # create a list for reference

## Data exploration

We want to explore the distribution of our dataset

In [ ]:
skew_emp = pd.DataFrame ((df1.loc[ : , df1.columns ].skew(axis = 0, skipna = True)), columns=["Skew_kubios"])
skew_emp.index.name = "HRV metric" #give name to the index
skew_far = pd.DataFrame ((df2.loc[ : , df2.columns ].skew(axis = 0, skipna = True)), columns = ["Skew_ALPS"])
skew_far.index.name = "HRV metric" #give name to the index
skew_tot = skew_emp.merge(skew_far, on = "HRV metric")
skew_tot #skewness of the data

If the skewness is between -0.5 and 0.5, the data are fairly symmetrical
If the skewness is between -1 and – 0.5 or between 0.5 and 1, the data are moderately skewed
If the skewness is less than -1 or greater than 1, the data are highly skewed

In [ ]:
skew_tot.loc[ abs(skew_tot["Skew_kubios"]) > 0.5] # number of values moderately skewed for Kubios

In [ ]:
skew_tot.loc[ abs(skew_tot["Skew_kubios"]) > 1] # number of values highly skewed for Kubios

In [ ]:
skew_tot.loc[ abs(skew_tot["Skew_ALPS"]) > 0.5] # number of values moderately skewed for ALPS

In [ ]:
skew_tot.loc[ abs(skew_tot["Skew_ALPS"]) > 1] # number of values highly skewed for ALPS

### Evaluate the normallity of our dataset

We can also explore the distribution with Shapiro-Wilk test

In [ ]:
shap1 = []
s_val_p1 = []
colnew_s1 =[] #create empty lists to save the data
for column in df1.columns: #iterate in our reference list
        x = df1.filter(like=column) #filter from the combine dataset only the columns that start with the "name" at a time
        x = x.dropna() #drop the NaN for the specified duplet
        if len(x) > 2: # setting a threshold of at least 3 values
            r, p = stats.shapiro(x) #shapiro test
            shap1.append (r)
            s_val_p1.append (p)
            colnew_s1.append (column) # add the specified values to the list
res_shap1 = pd.DataFrame (shap1, columns=["Shap_statistics_kubios"])
res_shap2 = pd.DataFrame (s_val_p1, columns=["P-value(Shap_stats_kubios)"])
res_shap3 = pd.DataFrame (colnew_s1, columns=["Name"]) # create df from lists
shap_stats1 = pd.concat([res_shap1,res_shap2,res_shap3], axis =1) #merge into one


shap2 = []
s_val_p2 = []
colnew_s2 =[] #create empty lists to save the data
for column in df1.columns: #iterate in our reference list
        x = df2.filter(like=column) #filter from the combine dataset only the columns that start with the "name" at a time
        x = x.dropna() #drop the NaN for the specified duplet
        if len(x) > 2: # setting a threshold of at least 3 values
            r, p = stats.shapiro(x) #shapiro test
            shap2.append (r)
            s_val_p2.append (p)
            colnew_s2.append (column) # add the specified values to the list
res_shap4 = pd.DataFrame (shap2, columns=["Shap_statistics_ALPS"])
res_shap5 = pd.DataFrame (s_val_p2, columns=["P-value(Shap_stats_ALPS)"])
res_shap6 = pd.DataFrame (colnew_s2, columns=["Name"]) # create df from lists
shap_stats2 = pd.concat([res_shap4,res_shap5,res_shap6], axis =1) #merge into one


shap_stats_tot = pd.merge(shap_stats1, shap_stats2, how= "outer", on="Name") 
shap_stats_tot.set_index("Name", inplace = True)
shap_stats_tot
 #visualize

### Pearson correlation

In [ ]:
corr = []
p_val_p = []
colnew_p =[] #create empty lists to save the data
for column in df1.columns: #iterate in our reference list
        x = df12_merged.filter(like=column) #filter from the combine dataset only the columns that start with the "name" at a time
        x = x.dropna() #drop the NaN for the specified duplet
        if len(x) > 2: # setting a threshold of at least 3 values
                r, p = stats.pearsonr(x[x.columns[0]], x[x.columns[1]]) #Pearson correlation
                corr.append (r)
                p_val_p.append (p)
                colnew_p.append (column) # add the specified values to the list
res_per1 = pd.DataFrame (corr, columns=["Pearson_correlation_value"])
res_per2 = pd.DataFrame (p_val_p, columns=["P-value(Pear-corr)"])
res_per3 = pd.DataFrame (colnew_p, columns=["Name"]) # create df from lists
per_corr = pd.concat([res_per1,res_per2,res_per3], axis =1) #merge into one
per_corr #visualize

### Student t-test

In [ ]:
stut = []
p_val_s = []
colnew_s =[]#create empty lists to save the data
for column in df1.columns: #iterate in our reference list
        x = df12_merged.filter(like=column) #filter from the combine dataset only the columns that start with the "name" at a time
        x = x.dropna() #drop the NaN for the specified duplet
        if len(x) > 2: # setting a threshold of at least 3 values
                r, p = stats.ttest_rel(x[x.columns[0]], x[x.columns[1]]) #paired test
                stut.append (r)
                p_val_s.append (p)
                colnew_s.append (column)# add the specified values to the list
res_stut1 = pd.DataFrame (stut, columns=["T-test_value"])
res_stut2 = pd.DataFrame (p_val_s, columns=["P-value(T-Test)"])
res_stut3 = pd.DataFrame (colnew_s, columns=["Name"])# create df from lists
t_test = pd.concat([res_stut1,res_stut2,res_stut3], axis =1) #merge into one
t_test #visualize

### Wilcoxon Rank test

In [ ]:
wilc = []
p_val_w = []
colnew_w =[]#create empty lists to save the data
for column in df1.columns: #iterate in our reference list
        x = df12_merged.filter(like=column) #filter from the combine dataset only the columns that start with the "name" at a time
        x = x.dropna() #drop the NaN for the specified duplet
        if len(x) > 2: # setting a threshold of at least 3 values
            r, p = stats.wilcoxon(x[x.columns[0]], x[x.columns[1]], alternative='two-sided') #wilcoxon test
            wilc.append (r)
            p_val_w.append (p)
            colnew_w.append (column)# add the specified values to the list
res_wilc1 = pd.DataFrame (wilc, columns=["Wilconxon_stats"])
res_wilc2 = pd.DataFrame (p_val_w, columns=["P-value(Wilcoxon)"])
res_wilc3 = pd.DataFrame (colnew_w, columns=["Name"])# create df from lists
wilc_rank = pd.concat([res_wilc1,res_wilc2,res_wilc3], axis =1) #merge into one
wilc_rank #visualize

## Create a comparison complete table

In [ ]:
per_corr.set_index("Name", inplace=True)
t_test.set_index("Name", inplace=True) # set the index

In [ ]:
per_corr#visualize

In [ ]:
t_test #visualize

In [ ]:
mean_diff #visualize

In [ ]:
per_t_compared = pd.merge(per_corr, t_test, how= "outer", on="Name") 
#merge pearson df's using the combine column on name

In [ ]:
per_t_compared #visualize

In [ ]:
non_wilc_compared = pd.merge(per_t_compared, mean_diff, how= "outer", on="Name") 
#merge pearson df's using the combine column on name

In [ ]:
total_compared = pd.merge(non_wilc_compared, wilc_rank, how= "outer", on="Name") 
#merge pearson df's using the combine column on name

In [ ]:
total_compared #visualize

In [ ]:
total_compared.to_csv("path...comparison_table_ALPS.csv", decimal=',', sep=';', index = False)
#save

### Bland-Altman Plot

In [ ]:
font1 = {'family':'serif','color':'blue','size':20}

for column in df1.columns:
    d = df12_merged.filter(like=column)  #filter from the combine dataset only the columns that start with the "name" at a time
    d = d.dropna() #drop the NaN for the specified duplet
    if len(d) > 2:    # setting a threshold of at least 3 values
        #create Bland-Altman plot                  
        f, ax = plt.subplots(1, figsize = (8,5))
        sm.graphics.mean_diff_plot(d[d.columns[0]], d[d.columns[1]], ax = ax)        
        plt.title(column, fontdict = font1)
        #display Bland-Altman plot
        plt.show()